In [180]:
import torch
import torch.nn as nn 
import sys
import os 
from pathlib import Path 
from dataset import Translate_dataset
from torch.utils.data import DataLoader
import pandas as pd
import torch.nn.functional as F
from einops import repeat
from neural_net import EncoderRNN, Attn, LuongAttnDecoderRNN
import random
import matplotlib.pyplot as plt

In [2]:
class Vocab:
    def __init__(self, name=None):
        self.name = name
        self.word2idx = {"PAD": 0, "SOS": 1, "EOS": 2}
        self.idx2word = {1: "SOS", 0: "PAD", 2: "EOS"} #3 special keywords
        self.num_words = 3
        self.word_counts = {} #SOS, EOS, PAD

    def add_sentence(self, sentence):
        for word in sentence.split():
            self.add_word(word)
    def add_word(self, word):
        if word in self.word2idx.keys():
            self.word_counts[word] += 1
        else:
            self.word2idx[word] = self.num_words
            self.idx2word[self.num_words] = word
            self.word_counts[word] = 1 
            self.num_words += 1

class normalize_funcs:
    def __init__(self, rareword=None):
        self.punctuation = string.punctuation
        self.rareword = rareword
    def lower_case(self, text):
        return text.lower()

    def remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', self.punctuation))

    def remove_rareword(self, text):
        return ' '.join(word for word in text.split() if word not in self.rareword)
    def trimming(self, input, target, min_freq=3):
        counter = {}
        for arg in [input, target]:
            for line in arg:
                for word in line.split():
                    if word in counter:
                        counter[word] += 1
                    else:
                        counter[word] = 1

        #min freq = 3

        rare_words = []
        for key, value in counter.items():
            if value < min_freq:
                rare_words.append(key)
        #trimming
        new_input = []
        new_target = []
        for line1, line2 in zip(input, target):
            if word in line1 or word in line2:
                continue
            else:
                new_input.append(line1)
                new_target.append(line2)
        return new_input, new_target

vocab = Vocab()
data = pd.read_csv(os.path.join('/home/xps/educate/code/NLP/chat_bot/data/cornell movie-dialogs corpus/pair_df.csv'), sep='@')
print(data)
for i in range(len(data)):
    input, target = data.iloc[i, :]
    for line in [input, target]:
        vocab.add_sentence(line)
print('VOCAB', vocab.name)
print('NUMWORDS', vocab.num_words)

                                                   input  \
0             gosh if only we could find kat a boyfriend   
1                           cesc ma tete this is my head   
2                                                  there   
3                       you have my word  as a gentleman   
4                                                     hi   
...                                                  ...   
64661                         what oclock is it mr noggs   
64662     are you dictating the strategy of this war sir   
64663                                             stuart   
64664  well fed or hungry pulleine wants them in posi...   
64665   do you think she might be interested in  someone   

                                         target  
0                      let me see what i can do  
1          right  see  youre ready for the quiz  
2                                         where  
3                                   youre sweet  
4      looks like things work

In [165]:
    vocab = Vocab()
    data = pd.read_csv(os.path.join('/home/xps/educate/code/NLP/chat_bot/data/cornell movie-dialogs corpus/pair_df.csv'), sep='@')
    data = data.iloc[:3000]
    print(data)
    for i in range(len(data)):
        input, target = data.iloc[i, :]
        vocab.add_sentence(input)
        vocab.add_sentence(target)
    dataset = Translate_dataset(vocab, data)
    print('---SHAPE', dataset[0][0].shape)
    #loader 
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    sample_input, sample_target, lengths, mask = iter(dataloader).next()
    lengths, _ = lengths.sort(descending=True)
    print('---encoder input', sample_input.shape, sample_target.shape, lengths)
    #try the model 
    # Configure models
    attn_model = 'dot'
    #attn_model = 'general'
    #attn_model = 'concat'
    hidden_size = 500
    encoder_n_layers = 2
    decoder_n_layers = 2
    dropout = 0.1
    batch_size = 32 
    embedding = nn.Embedding(vocab.num_words, hidden_size)
    #define the model
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, vocab.num_words, decoder_n_layers, dropout)
    #encode
    encoder_output, encoder_hidden  = encoder(sample_input.reshape(11, 32), lengths)
    print(encoder_output.shape, encoder_hidden.shape)
    #decoder
    decoder_input = torch.tensor([[vocab.word2idx['SOS'] for i in range(32)]])
    print('--decode input', decoder_input.shape)
    encoder_hidden = repeat(encoder_hidden[-1], 'b h -> n b h', n=decoder_n_layers)
    decoder_output, decoder_hidden = decoder(decoder_input, encoder_hidden, encoder_output)
    print('----decoder ouput', decoder_output.shape, decoder_hidden.shape)

                                           input  \
0     gosh if only we could find kat a boyfriend   
1                   cesc ma tete this is my head   
2                                          there   
3               you have my word  as a gentleman   
4                                             hi   
...                                          ...   
2995           in every sense if at all possible   
2996                         wheres the restroom   
2997              im going to be sick wheres the   
2998                                        what   
2999                          give me some money   

                                          target  
0                       let me see what i can do  
1           right  see  youre ready for the quiz  
2                                          where  
3                                    youre sweet  
4       looks like things worked out tonight huh  
...                                          ...  
2995  you cant hur

In [86]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to('cpu')
    return loss, nTotal.item()

In [79]:
sample_target.shape

torch.Size([32, 11])

In [87]:
maskNLLLoss(decoder_output, sample_target, mask)

(tensor(8.2910, grad_fn=<MeanBackward0>), 182)

In [129]:
input = decoder_output
target = sample_target
torch.log(torch.gather(input, 1, target.view(-1, 1)).squeeze(1)).shape

RuntimeError: Size does not match at dimension 0 expected index [352, 1] to be smaller than self [32, 4039] apart from dimension 1

In [84]:
input.shape, target.shape
torch.log(torch.gather(input, 1, target)).shape

torch.Size([32, 11])

In [137]:
out = torch.rand(32, 1000)
print(torch.argmax(out, dim=1).shape)

torch.Size([32])


In [69]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder,
          encoder_optimizer, decoder_optimizer, batch_size, clip, device='cpu'):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[1 for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            # print('TURN', t)
            # print('hidden shape', decoder_hidden.shape)
            # print('input', decoder_input.shape)
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable.reshape(-1, batch_size)[t].unsqueeze(0)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print(mask_loss)
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        # print('????')
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # # Clip gradients: gradients are modified in place
    # _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    # _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [173]:
def train(input_variable, lengths, target_variable, mask, encoder, decoder,
          encoder_optimizer, decoder_optimizer, batch_size, clip, device='cpu'):

      #input and optimizer
      if device == 'cuda':
        input_variable, target_variable = input_variable.to(device), target_variable.to(device)
        encoder.cuda()
        decoder.cuda()
      decoder_input = torch.tensor([[1] for i in range(batch_size)], dtype=torch.long).reshape(1, -1)
      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad() 

      #feed to model
      #encoder:
      encoder_output, encoder_hidden = encoder(input_variable, lengths)
      #repeat the final hidden according to the num_layers of encoder 
      decoder_hidden = repeat(encoder_hidden[-1], 'b h -> n b h', n=decoder.n_layers)

      total_loss = 0
      n_total = 0
      #target shape 11 x 32 (seq x batch)
      teacher_forcing = random.uniform(0, 1) > 0.5
      for i in range(target_variable.shape[0]): 
          decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
          if teacher_forcing:
            decoder_input = target_variable[i].reshape(1, batch_size) #target seq x batch
          else:
            decoder_input = torch.argmax(decoder_output, dim=1).reshape(1, batch_size) # seq 1 x batch 32 
  
          loss, ntotal = maskNLLLoss(decoder_output, target_variable[i].reshape(batch_size, 1), mask)
          total_loss += loss
          n_total += ntotal
      
      #backward
      loss.backward()

      #step
      encoder_optimizer.step()
      decoder_optimizer.step()

      return total_loss/n_total

        
            
           
            


        
    
      

In [163]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500
sample_input = sample_input.reshape(-1, 32)
sample_target = sample_target.reshape(-1, 32)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
# train(sample_input, lengths, sample_target, mask, encoder, decoder, encoder_optimizer, decoder_optimizer, 32, clip)

In [184]:
total_loss = []
for batch in dataloader:
    sample_input, sample_target, lengths, mask = batch
    if sample_input.shape[0] == 30:
        
        sample_input = sample_input.reshape(-1, 30)
        sample_target = sample_target.reshape(-1, 30)
        total_loss.append(train(sample_input, lengths, sample_target, mask, encoder, decoder, encoder_optimizer, decoder_optimizer, 30, clip).detach().numpy())
    else:
        print(sample_input.shape[0])
        continue


In [174]:
dataloader = DataLoader(dataset, batch_size=30, shuffle=True)

In [185]:
encoder.eval()
decoder.eval()

LuongAttnDecoderRNN(
  (embedding): Embedding(4039, 500)
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
  (concat): Linear(in_features=1000, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=4039, bias=True)
  (attn): Attn()
)

In [207]:
def evaluate(encoder, decoder, input_variable, lengths, MAX_LENGTH=11):
    decoder_input = torch.tensor([1], dtype=torch.long).reshape(1, 1)
    encoder_output, encoder_hidden = encoder(input_variable, lengths)
    decoder_hidden = repeat(encoder_hidden[-1], 'b h -> n b h', n=decoder.n_layers)
    print(encoder_output.shape, decoder_hidden.shape)
    out_token = []
    for i in range(MAX_LENGTH):
        decoder_ouput, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
        out_token.append(torch.argmax(decoder_ouput, dim=1))
        decoder_input = torch.argmax(decoder_ouput, dim=1).reshape(1, 1)
    
    return out_token

    


In [195]:
word = 'hello my name is'


word_tensor = torch.tensor([vocab.word2idx[i] for i in word.split()] + [vocab.word2idx['EOS']], dtype=torch.long)
word_tensor

tensor([1130,   24, 1058,   23,    2])

In [196]:
if len(word.split()) < 11:
          word_tensor = torch.cat([word_tensor, torch.zeros(10 - len(input.split()), dtype=torch.long)])

In [201]:
word_tensor = word_tensor.reshape(-1, 1)
word_tensor.shape

torch.Size([11, 1])

In [209]:
out_token = evaluate(encoder, decoder, word_tensor, torch.tensor([len(word.split())]))

torch.Size([4, 1, 500]) torch.Size([2, 1, 500])


In [212]:
sentence = ''
for i in out_token:
    sentence += vocab.idx2word[i.item()] + ' '

In [213]:
sentence

'learn learn days iksetkiba one one one one one approaching one '